# *get_fence_array(patch_length, overall_length)* revisited
```python
def get_fence_array_from_padded_segment_with_stride(patch_len, start_seg, end_seg, fill_complete=False):
    
    return fence_array
```

## Compare padding vs stride in the context of registration intersection pairs

In [46]:
import numpy as np

In [49]:
def get_strided_fence_array_NB(patch_length, start_seg, end_seg):
    """ Usage: fence_array = get_fence_array(patch_length, overall_length)
        create a left-right set of pairs that descrete overall_length into patch_length segments

    Args:
        patch_length:   patch size - number of pixels high or wide
        patch_length:   overall number of pixels high or wide

    Returns:
        fence_array:    boundry values for each segment
        -----------:    [[left_0, right_0],
                         [left_1, right_1],
                         [left_2, right_2],... ]
    """    
    # Determine the array size
    overall_length = end_seg - start_seg
    print('overall_length:', overall_length)
    n_fenced = np.ceil(overall_length / patch_length).astype(int)  # number of boxes
    stride = np.floor(overall_length / n_fenced).astype(int)
    fence_array = np.zeros((n_fenced, 2)).astype(int)
    
    print('stride', stride, 'n_fenced * stride', n_fenced * stride)
    fence_array[0,0] = start_seg
    fence_array[0,1] = start_seg + patch_length - 1

    for k in range(1, fence_array.shape[0]):    
        fence_array[k, 0] = fence_array[k-1, 0] + stride
        fence_array[k, 1] = fence_array[k, 0] + patch_length - 1
        
    fence_array[k, 1] = overall_length - 1
    fence_array[k, 0] = overall_length - patch_length
        
    return fence_array

patch_length = 224
remainder = 23
overall_length = 224*12 + remainder
start_seg = 22
end_seg = overall_length - 211

print('patch_length', patch_length, 'start_segment', start_seg, 'end_segment %i\n'%(end_seg))
fa = get_strided_fence_array_NB(patch_length, start_seg, end_seg)
print('Array Size =  %i\n'%(len(fa)))
for f in fa:
    print(f, '\t%i'%(f[1] - f[0]))


patch_length 224 start_segment 22 end_segment 2500

overall_length: 2478
stride 206 n_fenced * stride 2472
Array Size =  12

[ 22 245] 	223
[228 451] 	223
[434 657] 	223
[640 863] 	223
[ 846 1069] 	223
[1052 1275] 	223
[1258 1481] 	223
[1464 1687] 	223
[1670 1893] 	223
[1876 2099] 	223
[2082 2305] 	223
[2254 2477] 	223


```sh
# patch_length 224 overall_length 2711 remainder 23

[ 12 235]
[236 459]
[460 683]
[684 907]
[ 908 1131]
[1132 1355]
[1356 1579]
[1580 1803]
[1804 2027]
[2028 2251]
[2252 2475]
[2476 2699]
```

In [48]:
# patch_length 224 overall_length 2711 remainder 23

def get_fence_array_NB(patch_length, overall_length):
    """ Usage: fence_array = get_fence_array(patch_length, overall_length)
        create a left-right set of pairs that descrete overall_length into patch_length segments

    Args:
        patch_length:   patch size - number of pixels high or wide
        patch_length:   overall number of pixels high or wide

    Returns:
        fence_array:    boundry values for each segment
        -----------:    [[left_0, right_0],
                         [left_1, right_1],
                         [left_2, right_2],... ]
    """
    # Determine the array size
    n_fenced = overall_length // patch_length  # number of boxes
    n_remain = 1 + overall_length % patch_length  # number of pixels leftover
    paddit = n_remain // 2  # padding for the beginning

    if n_remain == patch_length:  # exact fit special case: exactly one left over
        paddit = 0
        n_fenced = n_fenced + 1

    # Allocate as integers for use as indices
    fence_array = np.zeros((n_fenced, 2)).astype(int)
    for k in range(n_fenced):
        # for each box edge, get the beginning and end pixel location
        if k == 0:
            # first case special (padding)
            fence_array[k, 0] = paddit
            # add the width to it
            fence_array[k, 1] = fence_array[k, 0] + patch_length - 1

        elif fence_array[k - 1, 1] + patch_length <= overall_length:
            # Previous right pixel plus one
            fence_array[k, 0] = fence_array[k - 1, 1] + 1
            # add the width to it
            fence_array[k, 1] = fence_array[k, 0] + patch_length - 1

    return fence_array

print('patch_length', patch_length, 'overall_length', overall_length, 'remainder %i\n'%(remainder))
fa_old = get_fence_array_NB(patch_length, overall_length)
print('Array Size =  %i\n'%(len(fa_old)))
for f0 in fa_old:
    print(f0, '\t%i'%(f0[1] - f0[0]))


patch_length 224 overall_length 2711 remainder 23

Array Size =  12

[ 12 235] 	223
[236 459] 	223
[460 683] 	223
[684 907] 	223
[ 908 1131] 	223
[1132 1355] 	223
[1356 1579] 	223
[1580 1803] 	223
[1804 2027] 	223
[2028 2251] 	223
[2252 2475] 	223
[2476 2699] 	223
